First we will install Transformer from hugging face!

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.8 MB/s 
     |████████████████████████████████| 6.6 MB 60.4 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Connecting with my drive!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing required dependencies!

In [ ]:
import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 11.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.5 MB/s 
     |████████████████████████████████| 212 kB 63.5 MB/s 
     |████████████████████████████████| 140 kB 57.6 MB/s 
     |████████████████████████████████| 127 kB 41.2 MB/s 
     |████████████████████████████████| 144 kB 54.5 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 271 kB 49.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from datasets import Dataset

we define if we are going to run the models on CPU or GPU. For this, we can check using torch if CUDA is installed, in which case, the GPU would be used. If it is not available, we leave CPU by default:

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)  

In [ ]:
device

device(type='cuda', index=0)

Loading Model gpt

In [ ]:
# We load the model for title
base_model_title = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
# We load the model for description
base_model_description = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
base_model_title.num_parameters

<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0

In [ ]:
base_model_description.num_parameters

<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0

Loading tokenizer from gpt

In [ ]:

# We load the tokenizer
base_tokenizer_title = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
# We load the tokenizer
base_tokenizer_description = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:

print('Words in vocabulary: ', base_tokenizer_title.vocab_size)

Words in vocabulary:  50257


In [ ]:
vocabulary = base_tokenizer_title.get_vocab()
vocabulary['Hi']

17250

Tokenizing text

In [ ]:
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}

# the new token is added to the tokenizer
num_added_toks = base_tokenizer_title.add_special_tokens(special_tokens_dict)

# the model config to which we add the special tokens
config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=base_tokenizer_title.bos_token_id,
                                    eos_token_id=base_tokenizer_title.eos_token_id,
                                    pad_token_id=base_tokenizer_title.pad_token_id,
                                    output_hidden_states=False)

# the pre-trained model is loaded with the custom configuration
base_model_title = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

# the model embedding is resized
base_model_title.resize_token_embeddings(len(base_tokenizer_title))

Embedding(50259, 768)

Loading dataset

In [ ]:
df_title = pd.read_csv("/content/dataset.csv")

pd.set_option("display.max_colwidth", None)
df_title.head(5)

,query,title,description
0,French quarter new York,the neighborhoods north of new York where the French live,the most street populated northern new York French neighborhoods the French and French speaking population of new York has always had a soft spot for the northern part of the region
1,French quarter new York,and the neighborhoods of new York that speak the most street French are,it is the dominant language after English in most street new York neighborhoods French dominates in midtown east street Murray hill
2,French quarter new York,Carroll gardens the favorite district of the French new York,partner article you could almost street call it little France Brooklyn Carroll gardens neighborhood never ceases to seduce
3,French quarter new York,where is the French quarter new York forum new York,I saw about months ago in the ft newspaper a report on a French neighborhood in Brooklyn where is he exactly is it
4,French quarter new York,is there a French quarter new York,the French quarter is the upper east street side near the French school after often it is near public schools that have a curriculum


our dataset is clean, we can add the start and end tokens to the headlines. And then we separate the dataset in training and validation

In [ ]:


def process_headlines_title(df_title, text_colname_title):
  
    # Remove empty and null rows
    titulo_vacio = (df_title['title'].str.len() == 0) | df_title['title'].isna()
    df_title = df_title[~titulo_vacio]

    # Remove title with less than 1 words
    titlos_len_ge8_title = (df_title['title'].str.split().apply(lambda x: len(x)) >= 1)
    df_title = df_title[titlos_len_ge8_title]

    # Drop duplicates
    text_df_title = df_title.drop_duplicates(subset = [text_colname_title])\
                [[text_colname_title]]

    return text_df_title
    
df_title = process_headlines_title(df_title, 'title')

In [ ]:
df_title['title'] = bos + ' ' + df_title['title'] + ' ' + eos

df_train_title, df_val_title = train_test_split(df_title, train_size = 0.9, random_state = 77)
print(f'There are {len(df_train_title)} headlines for training and {len(df_val_title)} for validation')

There are 1377 headlines for training and 154 for validation


In [ ]:
import pandas as pd

In [ ]:
# we load the datasets directly from a pandas df
train_dataset_title = Dataset.from_pandas(df_train_title[['title']])
val_dataset_title = Dataset.from_pandas(df_val_title[['title']])

In [ ]:
def tokenize_function_title(examples):
        return base_tokenizer_title(examples['title'], padding=True)


tokenized_train_dataset_title = train_dataset_title.map(
    tokenize_function_title,
    batched=True,
    num_proc=5,
    remove_columns=['title'],
)
tokenized_val_dataset_title = val_dataset_title.map(
    tokenize_function_title,
    batched=True,
    num_proc=5,
    remove_columns=['title'],
)

Parameter 'function'=<function tokenize_function_title at 0x7f18c24283b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Example of the result of the tokenization process with padding
base_tokenizer_title.decode(tokenized_train_dataset_title['input_ids'][0])

'<|endoftext|> airbnb in new york unique places best street prices <|EOS|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|>'

In [ ]:
model_title = '/content/drive/MyDrive/gpt/model_title'

training_args_title = TrainingArguments(
    output_dir=model_title,          # output directory
    num_train_epochs=20,              # total # of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=model_title,            # directory for storing logs
    prediction_loss_only=True,
    save_steps=10000 
)

In [ ]:
data_collator_title = DataCollatorForLanguageModeling(
        tokenizer=base_tokenizer_title,
        mlm=False
    )

In [ ]:
trainer_title = Trainer(
    model=base_model_title,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args_title,                  # training arguments, defined above
    data_collator=data_collator_title,
    train_dataset=tokenized_train_dataset_title,         # training dataset
    eval_dataset=tokenized_val_dataset_title            # evaluation dataset
)
trainer_title.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1377
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 880
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Step,Training Loss
500,7.824500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=880, training_loss=5.377123537930575, metrics={'train_runtime': 197.1206, 'train_samples_per_second': 139.711, 'train_steps_per_second': 4.464, 'total_flos': 323232532992000.0, 'train_loss': 5.377123537930575, 'epoch': 20.0})

In [ ]:
trainer_title.save_model()
base_tokenizer_title.save_pretrained(model_title)

Saving model checkpoint to /content/drive/MyDrive/gpt/model_title
Configuration saved in /content/drive/MyDrive/gpt/model_title/config.json
Model weights saved in /content/drive/MyDrive/gpt/model_title/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/gpt/model_title/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/gpt/model_title/special_tokens_map.json
added tokens file saved in /content/drive/MyDrive/gpt/model_title/added_tokens.json


('/content/drive/MyDrive/gpt/model_title/tokenizer_config.json',
 '/content/drive/MyDrive/gpt/model_title/special_tokens_map.json',
 '/content/drive/MyDrive/gpt/model_title/vocab.json',
 '/content/drive/MyDrive/gpt/model_title/merges.txt',
 '/content/drive/MyDrive/gpt/model_title/added_tokens.json')

In [ ]:
trainer_title.evaluate()

***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


{'epoch': 20.0,
 'eval_loss': 3.930478572845459,
 'eval_runtime': 0.3621,
 'eval_samples_per_second': 425.308,
 'eval_steps_per_second': 27.617}

In [ ]:
def generate_n_text_samples_title(model, tokenizer, input_text, device, n_samples = 10):
    text_ids = tokenizer.encode(input_text, return_tensors = 'pt')
    text_ids = text_ids.to(device)
    model = model.to(device)

    generated_text_samples = model.generate(
        text_ids, 
        max_length= 100,  
        num_return_sequences= n_samples,
        no_repeat_ngram_size= 2,
        repetition_penalty= 1.5,
        top_p= 0.92,
        temperature= .85,
        do_sample= True,
        top_k= 125,
        early_stopping= True
    )
    gen_text = []
    for t in generated_text_samples:
        text = tokenizer.decode(t, skip_special_tokens=True)
        gen_text.append(text)

        return gen_text

In [ ]:
# trained model loading
model_title_path = '/content/drive/MyDrive/gpt/model_title'


title_model = GPT2LMHeadModel.from_pretrained(model_title_path)
title_tokenizer = GPT2Tokenizer.from_pretrained(model_title_path)

#device = "cuda:0"

input_text = title_tokenizer.bos_token

title = generate_n_text_samples_title(title_model, title_tokenizer, 
                                    input_text, device, n_samples = 30)
#for h in title:
 #   print(h)
    #print()

loading configuration file /content/drive/MyDrive/gpt/model_title/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50257,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50258,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "use_cach

In [ ]:
print(title)

[' a rainy day new york tatsunis']


In [ ]:
def pretty_print(text, max_len_line=10):
    words = text.split(' ')
    len_line = 0
    line = ''
    for w in words:
        if w == '\n':
            print(line)
            line = ''
            continue
        if (len(line) + len(w)) > max_len_line:
            print(line)
            line = ''
        line += ' ' + w
    print(line)

In [ ]:
prompt = 'apartment for rent'
texts_title = generate_n_text_samples_title(base_model_title, base_tokenizer_title, prompt, device, n_samples =5)
for t in texts_title:
    pretty_print(t)
    print()

 apartment
 for rent
 in new
 york city



Description

In [ ]:
df_description = pd.read_csv('/content/dataset.csv')

pd.set_option("display.max_colwidth", None)
df_description.head(5)

,query,title,description
0,French quarter new York,the neighborhoods north of new York where the French live,the most street populated northern new York French neighborhoods the French and French speaking population of new York has always had a soft spot for the northern part of the region
1,French quarter new York,and the neighborhoods of new York that speak the most street French are,it is the dominant language after English in most street new York neighborhoods French dominates in midtown east street Murray hill
2,French quarter new York,Carroll gardens the favorite district of the French new York,partner article you could almost street call it little France Brooklyn Carroll gardens neighborhood never ceases to seduce
3,French quarter new York,where is the French quarter new York forum new York,I saw about months ago in the ft newspaper a report on a French neighborhood in Brooklyn where is he exactly is it
4,French quarter new York,is there a French quarter new York,the French quarter is the upper east street side near the French school after often it is near public schools that have a curriculum


In [ ]:

def process_description(df_description, text_colname):
  
    # Remove empty and null rows
    titulo_vacio_description = (df_description['description'].str.len() == 0) | df_description['description'].isna()
    df_description = df_description[~titulo_vacio_description]

    # Remove description with less than 8 words
    titlos_len_ge8_description = (df_description['description'].str.split().apply(lambda x: len(x)) >= 8)
    df_description = df_description[titlos_len_ge8_description]

    # Drop duplicates
    text_df_description = df_description.drop_duplicates(subset = [text_colname])\
                [[text_colname]]

    return text_df_description   

df_description = process_description(df_description, 'description')

In [ ]:
df_description['description'] = bos + ' ' + df_description['description'] + ' ' + eos

df_train_description, df_val_description = train_test_split(df_description, train_size = 0.9, random_state = 77)
print(f'There are {len(df_train_description)} headlines for training and {len(df_val_description)} for validation')

There are 1909 headlines for training and 213 for validation


In [ ]:
# we load the datasets directly from a pandas df
train_dataset_description = Dataset.from_pandas(df_train_description[['description']])
val_dataset_description = Dataset.from_pandas(df_val_description[['description']])

In [ ]:
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}

# the new token is added to the tokenizer
num_added_toks = base_tokenizer_description.add_special_tokens(special_tokens_dict)

# the model config to which we add the special tokens
config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=base_tokenizer_description.bos_token_id,
                                    eos_token_id=base_tokenizer_description.eos_token_id,
                                    pad_token_id=base_tokenizer_description.pad_token_id,
                                    output_hidden_states=False)

# the pre-trained model is loaded with the custom configuration
base_model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

# the model embedding is resized
base_model.resize_token_embeddings(len(base_tokenizer_description))

Assigning <|EOS|> to the eos_token key of the tokenizer
Adding <|EOS|> to the vocabulary
Assigning <|endoftext|> to the bos_token key of the tokenizer
Assigning <|pad|> to the pad_token key of the tokenizer
Adding <|pad|> to the vocabulary
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50257,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50258,
  "reorder_and_upcast_attn": false,
  "resid_pdrop

Embedding(50259, 768)

In [ ]:
def tokenize_function_description(examples):
        return base_tokenizer_description(examples['description'], padding=True)


tokenized_train_dataset_description = train_dataset_description.map(
    tokenize_function_description,
    batched=True,
    num_proc=5,
    remove_columns=['description'],
)
tokenized_val_dataset_description = val_dataset_description.map(
    tokenize_function_description,
    batched=True,
    num_proc=5,
    remove_columns=['description'],
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Example of the result of the tokenization process with padding
base_tokenizer_description.decode(tokenized_train_dataset_description['input_ids'][0])

'<|endoftext|> top places to watch a new york sunset hudson river park brooklyn bridge park the hudson river ferry <|EOS|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|>'

In [ ]:
model_description = '/content/drive/MyDrive/gpt/model_description'

training_args_description = TrainingArguments(
    output_dir=model_description,          # output directory
    num_train_epochs=20,              # total # of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=model_description,            # directory for storing logs
    prediction_loss_only=True,
    save_steps=10000 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator_description = DataCollatorForLanguageModeling(
        tokenizer=base_tokenizer_description,
        mlm=False
    )

In [ ]:
trainer_description = Trainer(
    model=base_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args_description,                  # training arguments, defined above
    data_collator=data_collator_description,
    train_dataset=tokenized_train_dataset_description,         # training dataset
    eval_dataset=tokenized_val_dataset_description            # evaluation dataset
)
trainer_description.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1909
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1200
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Step,Training Loss
500,8.842100
1000,2.710000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1200, training_loss=5.209517567952474, metrics={'train_runtime': 704.3931, 'train_samples_per_second': 54.203, 'train_steps_per_second': 1.704, 'total_flos': 1324791429120000.0, 'train_loss': 5.209517567952474, 'epoch': 20.0})

In [ ]:
trainer_description.save_model()
base_tokenizer_description.save_pretrained(model_description)

Saving model checkpoint to /content/drive/MyDrive/gpt/model_description
Configuration saved in /content/drive/MyDrive/gpt/model_description/config.json
Model weights saved in /content/drive/MyDrive/gpt/model_description/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/gpt/model_description/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/gpt/model_description/special_tokens_map.json
added tokens file saved in /content/drive/MyDrive/gpt/model_description/added_tokens.json


('/content/drive/MyDrive/gpt/model_description/tokenizer_config.json',
 '/content/drive/MyDrive/gpt/model_description/special_tokens_map.json',
 '/content/drive/MyDrive/gpt/model_description/vocab.json',
 '/content/drive/MyDrive/gpt/model_description/merges.txt',
 '/content/drive/MyDrive/gpt/model_description/added_tokens.json')

In [ ]:
trainer_description.evaluate()

***** Running Evaluation *****
  Num examples = 213
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


{'epoch': 20.0,
 'eval_loss': 4.054355144500732,
 'eval_runtime': 1.0536,
 'eval_samples_per_second': 202.169,
 'eval_steps_per_second': 13.288}

In [ ]:
def generate_n_text_samples_description(model, tokenizer, input_text, device, n_samples = 5):
    text_ids = tokenizer.encode(input_text, return_tensors = 'pt')
    text_ids = text_ids.to(device)
    model = model.to(device)

    generated_text_samples = model.generate(
        text_ids, 
        max_length= 100,  
        num_return_sequences= n_samples,
        no_repeat_ngram_size= 2,
        repetition_penalty= 1.5,
        top_p= 0.92,
        temperature= .85,
        do_sample= True,
        top_k= 125,
        early_stopping= True
    )
    gen_text = []
    for t in generated_text_samples:
        text = base_tokenizer_description.decode(t, skip_special_tokens=True)
        gen_text.append(text)

        return gen_text

In [ ]:
# trained model loading
model_description = '/content/drive/MyDrive/gpt/model_description'


description_model = GPT2LMHeadModel.from_pretrained(model_description)
description_tokenizer = GPT2Tokenizer.from_pretrained(model_description)

#device = "cuda:0"

input_text = description_tokenizer.bos_token

headlines = generate_n_text_samples_description(description_model, description_tokenizer, 
                                    input_text, device, n_samples = 5)
for h in headlines:
    print(h)
    print()

loading configuration file /content/drive/MyDrive/gpt/model_description/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50257,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50258,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "us

 for rent or sale in new york city from night find unique places to street stay with local host streets around the world rating votes



In [ ]:
def pretty_print(text, max_len_line=100):
    words = text.split(' ')
    len_line = 0
    line = ''
    for w in words:
        if w == '\n':
            print(line)
            line = ''
            continue
        if (len(line) + len(w)) > max_len_line:
            print(line)
            line = ''
        line += ' ' + w
    print(line)

In [ ]:
prompt = 'apartment for rent'
texts = generate_n_text_samples_description(base_model, base_tokenizer_description, prompt, device, n_samples = 5)
for t in texts:
    pretty_print(t)
    print()

 apartment for rent in new york is on average more expensive than the typical monthly apartment
 rental price a month however this does not mean that it remains flat



In [ ]:
from numpy import promote_types
import pandas as pd

df = pd.DataFrame()
prompt = ['apartment for rent', 'New york apartment', 'New york rent Airbnb', 'furnished apartments', 'luxury real estate', ' shared rooms apartments', 'top new york sights', 'cheap room rent new york', ' house for sale', 'cheap flight'
          'airbnb in new york', 'new york streetock exchange', 'buy an apartment new york', 'a rainy day new york', 'new york skyscrapers', 'cheap new york accommodation', 'new york in winter', 'new york night club', 'new york building', 'famous new york street']
description = generate_n_text_samples_description(base_model, base_tokenizer_description, prompt, device)

#for i in prompt:
 # print(i)


for i in prompt:
  #print('keyword--->', i)
  for j in range(50):
    descript = generate_n_text_samples_description(base_model, base_tokenizer_description, i, device)
    title = generate_n_text_samples_title(base_model_title, base_tokenizer_title, i, device)
    print(i,',', j+1, title[0],',', j+1,  descript[0])
    #print("Description", j+1,  descript[0])
    

apartment for rent , 1 apartment for rent in new york rentals , 1 apartment for rent in new york street state thu apt du monde the apartment rental unit is located at weast streetseast floor of e wed sun
apartment for rent , 2 apartment for rent in new york city , 2 apartment for rent in new york or bowery co ugation the one time missing
apartment for rent , 3 apartment for rent in new york trulia , 3 apartment for rent rental unit in new york this streetore is located on th avenue between bleecker st and
apartment for rent , 4 apartment for rent in new york street state , 4 apartment for rent in new york city with wifi and water located at bowery manhattan nolita riverfront area will be landscaped like a condo
apartment for rent , 5 apartment for rent in new york rent city , 5 apartment for rent in new york with roommates and friends usa rental unit located at weast street th ave apartments is equipped toreeter
apartment for rent , 6 apartment for rent in new york city , 6 apartment f

Loading model and generating text!